In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from utils import show_graph

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def train():
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    history = []
    iterep = 500
    for i in range(iterep * 30):
        x_train, y_train = mnist.train.next_batch(100)
        sess.run(train_step,
                 feed_dict={'x:0': x_train, 
                            'y:0': y_train, 
                            'phase:0': 1})
        if (i + 1) %  iterep == 0:
            epoch = (i + 1)/iterep
            tr = sess.run([loss, accuracy], 
                          feed_dict={'x:0': mnist.train.images,
                                     'y:0': mnist.train.labels,
                                     'phase:0': 1})
            t = sess.run([loss, accuracy], 
                         feed_dict={'x:0': mnist.test.images,
                                    'y:0': mnist.test.labels,
                                    'phase:0': 0})
            history += [[epoch] + tr + t]
            print (history[-1])
    return (history)

In [4]:
def dense(x, size, scope):
    return tf.contrib.layers.fully_connected(x, size, 
                                             activation_fn=None,
                                             scope=scope)

def dense_relu(x, size, scope):
    with tf.variable_scope(scope):
        h1 = dense(x, size, 'dense')
        return tf.nn.relu(h1, 'relu')

In [5]:
tf.reset_default_graph()
x = tf.placeholder('float32', (None, 784), name='x')
y = tf.placeholder('float32', (None, 10), name='y')
phase = tf.placeholder(tf.bool, name='phase')

h1 = dense_relu(x, 100, 'layer1')
h2 = dense_relu(h1, 100, 'layer2')
logits = dense(h2, 10, scope='logits')

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(y, 1), tf.argmax(logits, 1)), 
            'float32'))

with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
show_graph(tf.get_default_graph().as_graph_def())

In [7]:
history = train()

[1.0, 0.65424323, 0.83443636, 0.63041794, 0.84320003]
[2.0, 0.43585038, 0.87901819, 0.41456324, 0.88639998]
[3.0, 0.36735448, 0.89596361, 0.34979472, 0.90039998]
[4.0, 0.33057824, 0.90547276, 0.31587887, 0.90890002]
[5.0, 0.30599794, 0.91232729, 0.29181904, 0.9145]
[6.0, 0.28673548, 0.91836363, 0.27572477, 0.92180002]
[7.0, 0.2702997, 0.92303634, 0.26061752, 0.9249]
[8.0, 0.25792703, 0.92630911, 0.24979782, 0.92809999]
[9.0, 0.24532108, 0.93001819, 0.23786758, 0.9339]
[10.0, 0.23503238, 0.93223637, 0.22858691, 0.93440002]
[11.0, 0.22438034, 0.93612725, 0.21967454, 0.9382]
[12.0, 0.2157865, 0.93949091, 0.21348472, 0.94129997]
[13.0, 0.20796573, 0.94101816, 0.20625025, 0.94230002]
[14.0, 0.2004853, 0.94252729, 0.1975375, 0.94389999]
[15.0, 0.19349591, 0.94496363, 0.19125254, 0.94599998]
[16.0, 0.18703344, 0.94654548, 0.18684715, 0.9472]
[17.0, 0.18080378, 0.94858181, 0.18143047, 0.94959998]
[18.0, 0.1752864, 0.95087272, 0.17555583, 0.95039999]
[19.0, 0.17013471, 0.95158184, 0.17041843, 0